In [ ]:
import sqlite3
import pandas as pd
import glob

# Path where all your table CSVs live
csv_folder = "../data/raw/site_tables/"
db_path = "../data/processed/site_database.sqlite"

# Create SQLite connection
conn = sqlite3.connect(db_path)

# Loop through all CSV files
for file in glob.glob(f"{csv_folder}/*.csv"):
    table_name = file.split("/")[-1].replace(".csv", "")
    df = pd.read_csv(file)
    df.to_sql(table_name, conn, if_exists="replace", index=False)

conn.close()


In [ ]:
import pandas as pd
from collections import defaultdict

def load_schema_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    
    # Expecting columns like:
    # table_name, table_display_name, column_name, column_display_name, column_type
    
    schema = defaultdict(lambda: {"display_name": "", "columns": {}})
    
    for _, row in df.iterrows():
        table_name = row["table_name"]
        table_display = row["table_display_name"]
        col_name = row["column_name"]
        col_display = row["column_display_name"]
        col_type = row["column_type"]
        
        schema[table_name]["display_name"] = table_display
        schema[table_name]["columns"][col_name] = {
            "display_name": col_display,
            "type": col_type
        }
    
    return dict(schema)

# Example usage:
table_mappings = load_schema_from_csv("../data/raw/Site Data Dictionary.csv")

# save out table_mappings for later use
import json
with open("../data/processed/table_mappings.json", "w") as f:
    json.dump(table_mappings, f, indent=4)

In [ ]:
import dateparser
import dateparser.search
from datetime import datetime
from dateutil.relativedelta import relativedelta

def describe_relative_date(dt, reference=None):
    if reference is None:
        reference = datetime.now()

    delta = relativedelta(reference, dt)

    if delta.years > 0:
        return f"{delta.years} year{'s' if delta.years > 1 else ''} ago"
    elif delta.months > 0:
        return f"{delta.months} month{'s' if delta.months > 1 else ''} ago"
    elif delta.days >= 7:
        weeks = delta.days // 7
        return f"{weeks} week{'s' if weeks > 1 else ''} ago"
    elif delta.days > 0:
        return f"{delta.days} day{'s' if delta.days > 1 else ''} ago"
    else:
        return "today"

In [55]:
import pandas as pd
import json

def patient_data_to_json(patient_id, conn, table_mappings, tables_to_include=None):
    """
    Pull all available data for one patient across multiple tables,
    filter/rename columns using the table_mappings, and return as JSON.
    
    Args:
        patient_id (str): The patient identifier to query.
        conn: Database connection object (e.g., SQLAlchemy or sqlite3).
        table_mappings (dict): Metadata dict with table and column mappings.
    
    Returns:
        str: JSON string with patient data.
    """

    patient_dict = {}

    if tables_to_include is not None:
        table_mappings = {k: v for k, v in table_mappings.items() if k in tables_to_include}

    for table_name, mapping in table_mappings.items():
        columns = list(mapping["columns"].keys())
        query = f"SELECT {', '.join(columns)} FROM {table_name} WHERE patient_id = ?"
        df = pd.read_sql(query, conn, params=(patient_id,))

        if df.empty:
            continue  # skip tables with no rows

        # Build rename mapping: {column_name: display_name}
        rename_map = {col: col_info["display_name"] for col, col_info in mapping["columns"].items()}
        df = df.rename(columns=rename_map)

        # drop columns that are entirely null
        df = df.dropna(axis=1, how="all")

        for col in df.columns:
            if "date" in col.lower():
                df[col] = df[col].apply(lambda x: describe_relative_date(pd.to_datetime(x)) if pd.notnull(x) else x)

        # convert to dict format
        records = df.to_dict(orient="records")

        # single row table → dict
        display_name = mapping["display_name"]
        if len(records) == 1:
            patient_dict[display_name] = records[0]
        else:
            patient_dict[display_name] = records

    # convert to JSON
    return json.dumps(patient_dict, indent=2, default=str)


In [ ]:
conn = sqlite3.connect("../data/processed/site_database.sqlite")
patient_json = patient_data_to_json(3277, conn, table_mappings)
print(patient_json)

In [ ]:
table_descriptions = {
    "kenyaemr_etl_etl_clinical_encounter": {"description": "Patient diagnoses and prescriptions during clinical encounters."},
    "kenyaemr_etl_etl_ccc_defaulter_tracing": {"description": "Patient defaulter tracing information."},
    "kenyaemr_etl_etl_drug_event": {"description": "Drug regimens and history."},
    "kenyaemr_etl_etl_drug_order": {"description": "Drug orders and prescriptions."},
    "kenyaemr_etl_etl_enhanced_adherence": {"description": "Information on support provided to patients with adherence issues."},
    "kenyaemr_etl_etl_hiv_enrollment": {"description": "Information about patient enrollment in HIV treatment."},
    "kenyaemr_etl_etl_laboratory_extract": {"description": "Information on laboratory test results."},
    "kenyaemr_etl_etl_patient_demographics": {"description": "Information on patient demographics.."},
    "kenyaemr_etl_etl_patient_hiv_followup": {"description": "Information about routine visits from patients on HIV treatment."},
    "kenyaemr_etl_etl_patient_triage": {"description": "Information on patient vital signs, symptoms and complaints."},
    "kenyaemr_etl_etl_tb_screening": {"description": "Screening for tuberculosis in patients."},
    "openmrs_encounter_diagnosis": {"description": "Patient diagnoses."}
}


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
import json

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def build_table_selection_prompt():
    human_template = """
Question: "{question}"

Here are the patient tables and their descriptions:
{table_descriptions}

Return only the exact table names needed to answer the question, in JSON format like:
["Table Name 1", "Table Name 2"]

Do not include columns, explanations, or anything else.
"""
    human_message = HumanMessagePromptTemplate.from_template(human_template)
    prompt = ChatPromptTemplate.from_messages([human_message])
    return prompt

def get_relevant_tables(question, table_descriptions):
    prompt = build_table_selection_prompt()
    # Format with actual values
    messages = prompt.format_prompt(
        question=question,
        table_descriptions=json.dumps(table_descriptions, indent=2)
    ).to_messages()

    # Send to LLM
    response = llm(messages)
    
    try:
        tables = json.loads(response.content)
        tables = [t for t in tables if isinstance(t, str)]
    except json.JSONDecodeError:
        tables = []
    return tables


In [ ]:
question = "What was their last viral load result and date? Any history of elevated viral loads over 1000 copies/mL?"
relevant_tables = get_relevant_tables(question, table_descriptions)
print(relevant_tables)

In [ ]:
filtered_tables = {k: v for k, v in table_descriptions.items() if k in relevant_tables}

In [57]:
conn = sqlite3.connect("../data/processed/site_database.sqlite")
# patient_json = patient_data_to_json(3277, conn, filtered_tables)
# print(patient_json)

patient_json = patient_data_to_json(
    patient_id=9439,
    conn=conn,
    table_mappings=table_mappings,  # full metadata dict
    tables_to_include=filtered_tables     # LLM-filtered tables
)

print(patient_json)

{
  "Laboratory Testing Results": [
    {
      "Visit Date": "1 week ago",
      "Lab test": 2001197,
      "Urgency": "STAT",
      "Order test name": "TB LAM (Lipoarabino-mannan) Test"
    },
    {
      "Visit Date": "1 week ago",
      "Lab test": 162202,
      "Urgency": "STAT",
      "Order test name": "GeneXpert MTB/RIF"
    }
  ]
}
